# NLP Preprocessing

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai import *

`text.tranform` contains the functions that deal behind the scenes with the two main tasks when preparing texts for modelling: *tokenization* and *numericalization*.

*Tokenization* splits the raw texts into tokens (wich can be words, or punctuation signs...). The most basic way to do this would be to separate according to spaces, but it's possible to be more subtle; for instance, the contractions like "isn't" or "don't" should be split in \["is","n't"\] or \["do","n't"\]. By default fastai will use the powerful [spacy tokenizer](https://spacy.io/api/tokenizer).

*Numericalization* is easier as it just consists in attributing a unique id to each token and mapping each of those tokens to their respective ids.

## Tokenization

### Introduction

This step is actually divided in two phases: first, we apply a certain list of `rules` to the raw texts as preprocessing, then we use the tokenizer to split them in lists of tokens. Combining together those `rules`, the `tok_func`and the `lang` to process the texts is the role of the [`Tokenizer`](/text.transform.html#Tokenizer) class.

In [ ]:
show_doc(Tokenizer)

<h2 id="Tokenizer"><code>class</code> <code>Tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L84" class="source_link">[source]</a></h2>

> <code>Tokenizer</code>(`tok_func`:`Callable`=`'SpacyTokenizer'`, `lang`:`str`=`'en'`, `pre_rules`:`ListRules`=`None`, `post_rules`:`ListRules`=`None`, `special_cases`:`StrList`=`None`, `n_cpus`:`int`=`None`)

Put together rules and a tokenizer function to tokenize text with multiprocessing.  

This class will process texts by appling them the `pre_rules`, tokenizing them with `tok_func(lang)` and then applying the `post_rules`. `special_cases` are a list of tokens passed as special to the tokenizer and `n_cpus` is the number of cpus to use for multi-processing (by default, half the cpus available). We don't directly pass a tokenizer for multi-processing purposes: each process needs to initiate a tokenizer of its own. The rules and special_cases default to

`default_pre_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces]` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L80">[source]</a></div>

`default_post_rules = [replace_all_caps, deal_caps]` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L82">[source]</a></div>

and

`default_spec_tok = [BOS, FLD, UNK, PAD]` <div style="text-align: right"><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L81">[source]</a></div>

In [ ]:
show_doc(Tokenizer.process_text)

<h4 id="Tokenizer.process_text"><code>process_text</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L100" class="source_link">[source]</a></h4>

> <code>process_text</code>(`t`:`str`, `tok`:[`BaseTokenizer`](/text.transform.html#BaseTokenizer)) → `List`\[`str`\]

Process one text `t` with tokenizer `tok`.  

In [ ]:
show_doc(Tokenizer.process_all)

<h4 id="Tokenizer.process_all"><code>process_all</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L113" class="source_link">[source]</a></h4>

> <code>process_all</code>(`texts`:`StrList`) → `List`\[`List`\[`str`\]\]

Process a list of `texts`.  

For an example, we're going to grab some IMDB reviews.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

In [ ]:
df = pd.read_csv(path/'texts.csv', header=None)
example_text = df.iloc[2][1]; example_text

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [ ]:
tokenizer = Tokenizer()
tok = SpacyTokenizer('en')
' '.join(tokenizer.process_text(example_text, tok))

'xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively cheery . xxmaj there are no really superstars in the cast , though several faces will be familiar . xxmaj the entire cast does an excellent job with the script . \n\n xxmaj but it is hard to watch , because there is no good end to a situation like the one presented . xxmaj it is now fashionable to blame the xxmaj british for setting xxmaj hindus and xxmaj muslims against each other , and then cruelly separating them into two countries . xxmaj there is some merit in this view , but it \'s also true that no one forced xxmaj hindus and xxmaj muslims in the region to mistreat each other as they did around the time of partition . xxmaj it seems more likely that the xxmaj british simply saw the tensions between the religions and were clever enough to exploit them to their own e

As explained before, the tokenizer split the text according to words/punctuations signs but in a smart manner. The rules (see below) also have modified the text a little bit. We can tokenize a list of texts directly at the same time:

In [ ]:
df = pd.read_csv(path/'texts.csv', header=None)
texts = df[1].values
tokenizer = Tokenizer()
tokens = tokenizer.process_all(texts)
' '.join(tokens[2])

'xxmaj this is a extremely well - made film . xxmaj the acting , script and camera - work are all first - rate . xxmaj the music is good , too , though it is mostly early in the film , when things are still relatively cheery . xxmaj there are no really superstars in the cast , though several faces will be familiar . xxmaj the entire cast does an excellent job with the script . \n\n xxmaj but it is hard to watch , because there is no good end to a situation like the one presented . xxmaj it is now fashionable to blame the xxmaj british for setting xxmaj hindus and xxmaj muslims against each other , and then cruelly separating them into two countries . xxmaj there is some merit in this view , but it \'s also true that no one forced xxmaj hindus and xxmaj muslims in the region to mistreat each other as they did around the time of partition . xxmaj it seems more likely that the xxmaj british simply saw the tensions between the religions and were clever enough to exploit them to their own e

### Customize the tokenizer

The `tok_func` must return an instance of [`BaseTokenizer`](/text.transform.html#BaseTokenizer):

In [ ]:
show_doc(BaseTokenizer)

<h2 id="BaseTokenizer"><code>class</code> <code>BaseTokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L13" class="source_link">[source]</a></h2>

> <code>BaseTokenizer</code>(`lang`:`str`)

Basic class for a tokenizer function.  

In [ ]:
show_doc(BaseTokenizer.tokenizer)

<h4 id="BaseTokenizer.tokenizer"><code>tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L16" class="source_link">[source]</a></h4>

> <code>tokenizer</code>(`t`:`str`) → `List`\[`str`\]

Take a text `t` and returns the list of its tokens.

In [ ]:
show_doc(BaseTokenizer.add_special_cases)

<h4 id="BaseTokenizer.add_special_cases"><code>add_special_cases</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L17" class="source_link">[source]</a></h4>

> <code>add_special_cases</code>(`toks`:`StrList`)

Record a list of special tokens `toks`.

The fastai library uses [spacy](https://spacy.io/) tokenizers as its default. The following class wraps it as [`BaseTokenizer`](/text.transform.html#BaseTokenizer).

In [ ]:
show_doc(SpacyTokenizer)

<h2 id="SpacyTokenizer"><code>class</code> <code>SpacyTokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L19" class="source_link">[source]</a></h2>

> <code>SpacyTokenizer</code>(`lang`:`str`) :: [`BaseTokenizer`](/text.transform.html#BaseTokenizer)

Wrapper around a spacy tokenizer to make it a [`BaseTokenizer`](/text.transform.html#BaseTokenizer).  

If you want to use your custom tokenizer, just subclass the [`BaseTokenizer`](/text.transform.html#BaseTokenizer) and override its `tokenizer` and `add_spec_cases` functions.

### Rules

Rules are just functions that take a string and return the modified string. This allows you to customize the list of `default_pre_rules` or `default_post_rules` as you please. Those are:

In [ ]:
show_doc(deal_caps, doc_string=False)

<h4 id="deal_caps"><code>deal_caps</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L72" class="source_link">[source]</a></h4>

> <code>deal_caps</code>(`x`:`StrList`) → `StrList`

In `t`, every word is lower-casse. If a word begins with a capital, we put a token `TK_MAJ` in front of it.

In [ ]:
show_doc(fix_html, doc_string=False)

<h4 id="fix_html"><code>fix_html</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L55" class="source_link">[source]</a></h4>

> <code>fix_html</code>(`x`:`str`) → `str`

This rules replaces a bunch of HTML characters or norms in plain text ones. For instance `<br />` are replaced by `\n`, `&nbsp;` by spaces etc...

In [ ]:
fix_html("Some HTML&nbsp;text<br />")

'Some HTML& text\n'

In [ ]:
show_doc(replace_all_caps)

<h4 id="replace_all_caps"><code>replace_all_caps</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L64" class="source_link">[source]</a></h4>

> <code>replace_all_caps</code>(`x`:`StrList`) → `StrList`

Add `TK_UP` for words in all caps in `x`.  

In [ ]:
show_doc(replace_rep, doc_string=False)

<h4 id="replace_rep"><code>replace_rep</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L39" class="source_link">[source]</a></h4>

> <code>replace_rep</code>(`t`:`str`) → `str`

Whenever a character is repeated more than three times in `t`, we replace the whole thing by 'TK_REP n char' where n is the number of occurences and char the character.

In [ ]:
replace_rep("I'm so excited!!!!!!!!")

"I'm so excited xxrep 8 ! "

In [ ]:
show_doc(replace_wrep, doc_string=False)

<h4 id="replace_wrep"><code>replace_wrep</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L47" class="source_link">[source]</a></h4>

> <code>replace_wrep</code>(`t`:`str`) → `str`

Whenever a word is repeated more than four times in `t`, we replace the whole thing by 'TK_WREP n w' where n is the number of occurences and w the word repeated.

In [ ]:
replace_wrep("I've never ever ever ever ever ever ever ever done this.")

"I've never  xxwrep 7 ever  done this."

In [ ]:
show_doc(rm_useless_spaces)

<h4 id="rm_useless_spaces"><code>rm_useless_spaces</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L35" class="source_link">[source]</a></h4>

> <code>rm_useless_spaces</code>(`t`:`str`) → `str`

Remove multiple spaces in `t`.  

In [ ]:
rm_useless_spaces("Inconsistent   use  of     spaces.")

'Inconsistent use of spaces.'

In [ ]:
show_doc(spec_add_spaces)

<h4 id="spec_add_spaces"><code>spec_add_spaces</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L31" class="source_link">[source]</a></h4>

> <code>spec_add_spaces</code>(`t`:`str`) → `str`

Add spaces around / and # in `t`.  

In [ ]:
spec_add_spaces('I #like to #put #hashtags #everywhere!')

'I  # like to  # put  # hashtags  # everywhere!'

## Numericalization

To convert our set of tokens to unique ids (and be able to have them go through embeddings), we use the following class:

In [ ]:
show_doc(Vocab)

<h2 id="Vocab"><code>class</code> <code>Vocab</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L119" class="source_link">[source]</a></h2>

> <code>Vocab</code>(`itos`:`StrList`)

Contain the correspondance between numbers and tokens and numericalize.  

`itos` contains the id to token correspondance.

In [ ]:
show_doc(Vocab.create)

<h4 id="Vocab.create"><code>create</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L140" class="source_link">[source]</a></h4>

> <code>create</code>(`tokens`:`Tokens`, `max_vocab`:`int`, `min_freq`:`int`) → `Vocab`

Create a vocabulary from a set of `tokens`.  

Only keeps `max_vocab` tokens, and only if they appear at least `min_freq` times, set the rest to `UNK`.

In [ ]:
show_doc(Vocab.numericalize)

<h4 id="Vocab.numericalize"><code>numericalize</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L125" class="source_link">[source]</a></h4>

> <code>numericalize</code>(`t`:`StrList`) → `List`\[`int`\]

Convert a list of tokens `t` to their ids.  

In [ ]:
show_doc(Vocab.textify)

<h4 id="Vocab.textify"><code>textify</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L129" class="source_link">[source]</a></h4>

> <code>textify</code>(`nums`:`Collection`\[`int`\], `sep`=`' '`) → `List`\[`str`\]

Convert a list of `nums` to their tokens.  

In [ ]:
vocab = Vocab.create(tokens, max_vocab=1000, min_freq=2)
vocab.numericalize(tokens[2])[:10]

[14, 9, 6, 619, 85, 17, 110, 25, 4, 2]

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(SpacyTokenizer.tokenizer)

<h4 id="SpacyTokenizer.tokenizer"><code>tokenizer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L24" class="source_link">[source]</a></h4>

> <code>tokenizer</code>(`t`:`str`) → `List`\[`str`\]

In [ ]:
show_doc(SpacyTokenizer.add_special_cases)

<h4 id="SpacyTokenizer.add_special_cases"><code>add_special_cases</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/transform.py#L27" class="source_link">[source]</a></h4>

> <code>add_special_cases</code>(`toks`:`StrList`)

## New Methods - Please document or move to the undocumented section